<a href="https://colab.research.google.com/github/liunai7/CapstoneProjects/blob/master/text_embedding_cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install cohere altair -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [4]:
import cohere

co = cohere.ClientV2("Fxck3NWXDBYtEqiMUl4pKrgUEyN6DZGqA0qyBPPg") # Get your free API key: https://dashboard.cohere.com/api-keys


In [5]:
# Load the dataset to a dataframe
df_orig = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/atis_intents_train.csv', names=['intent','query'])

# Take a small sample for illustration purposes
sample_classes = ['atis_airfare', 'atis_airline', 'atis_ground_service']
df = df_orig.sample(frac=0.1, random_state=30)
df = df[df.intent.isin(sample_classes)]
df_orig = df_orig.drop(df.index)
df.reset_index(drop=True,inplace=True)

# Remove unnecessary column
intents = df['intent'] #save for a later need
df.drop(columns=['intent'], inplace=True)
df.head()

<ipython-input-5-6eb22591daf3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['intent'], inplace=True)


,query
0,which airlines fly from boston to washington ...
1,show me the airlines that fly between toronto...
2,show me round trip first class tickets from n...
3,i'd like the lowest fare from denver to pitts...
4,show me a list of ground transportation at bo...


In [7]:
df.shape

(91, 1)

In [8]:
df_orig.shape

(4743, 2)

In [9]:
df_orig.head()

,intent,query
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [10]:
# Get text embeddings
def get_embeddings(texts, model="embed-english-v3.0", input_type="search_document"):
    output = co.embed(
        texts=texts,
        model=model,
        input_type=input_type,
        embedding_types=["float"]
    )
    return output.embeddings.float

In [11]:
# Embed the dataset
df['query_embeds'] = get_embeddings(df['query'].tolist())
df.head()

,query,query_embeds
0,which airlines fly from boston to washington ...,"[0.02609253, 0.012168884, -0.008903503, 0.0114..."
1,show me the airlines that fly between toronto...,"[0.013801575, 0.017181396, -0.014984131, -0.00..."
2,show me round trip first class tickets from n...,"[0.02053833, -0.038482666, 0.061523438, 0.0099..."
3,i'd like the lowest fare from denver to pitts...,"[0.0016708374, 0.015625, -0.029022217, 0.03759..."
4,show me a list of ground transportation at bo...,"[0.037628174, -0.007888794, -0.0024662018, -0...."


In [12]:
# Function to return the principal components
def get_pc(arr, n):
    pca = PCA(n_components=n)
    embeds_transform = pca.fit_transform(arr)
    return embeds_transform

# Reduce embeddings to 10 principal components to aid visualization
embeds = np.array(df['query_embeds'].tolist())
embeds_pc = get_pc(embeds, 10)


In [13]:
embeds

array([[ 0.02609253,  0.01216888, -0.0089035 , ..., -0.00745392,
        -0.00959015,  0.00817108],
       [ 0.01380157,  0.0171814 , -0.01498413, ...,  0.015625  ,
        -0.02497864,  0.02479553],
       [ 0.02053833, -0.03848267,  0.06152344, ..., -0.03231812,
        -0.00874329, -0.00062275],
       ...,
       [ 0.02819824, -0.01511383,  0.00802612, ..., -0.01408386,
        -0.02952576,  0.00892639],
       [ 0.01974487,  0.01212311, -0.03164673, ..., -0.02539062,
        -0.04162598,  0.01139069],
       [ 0.03128052, -0.00934601, -0.00961304, ...,  0.00611496,
        -0.01441193,  0.02555847]])

In [14]:
embeds_pc

array([[ 7.37954373e-02, -2.52855056e-01,  1.74886820e-01,
         1.24960612e-01, -5.55749433e-02, -3.26577416e-03,
         3.65052113e-02, -6.40392615e-02, -1.29217037e-01,
        -3.76984886e-02],
       [ 5.04996216e-03, -3.16272869e-01, -2.97074336e-01,
        -1.37032946e-01, -1.70317669e-01, -9.72730334e-02,
         2.38999728e-02, -1.06202888e-01, -9.33228617e-02,
         2.29575388e-02],
       [ 1.95839721e-01,  8.11596042e-02, -3.44065893e-01,
         2.13443702e-01, -8.77724939e-02,  2.36043819e-02,
        -6.40465165e-03,  4.65869621e-02,  1.51199166e-02,
        -3.17021568e-01],
       [ 1.11741163e-01,  2.05993062e-01, -8.93970780e-02,
        -3.02112940e-01, -2.70444069e-01, -1.80696529e-01,
        -7.00367917e-02,  2.82606118e-01,  3.52967442e-02,
         2.51464408e-01],
       [-3.13675574e-01, -2.29584297e-01,  3.18650377e-02,
         1.48919222e-01, -1.34895814e-02, -1.84909060e-01,
        -7.40636403e-02,  2.78595396e-02, -6.02459099e-02,
        -9.

In [15]:
# Set sample size to visualize
sample = 9

# Reshape the data for visualization purposes
source = pd.DataFrame(embeds_pc)[:sample]
source = pd.concat([source,df['query']], axis=1)
source = source.melt(id_vars=['query'])

# Configure the plot
chart = alt.Chart(source).mark_rect().encode(
    x=alt.X('variable:N', title="Embedding"),
    y=alt.Y('query:N', title='',axis=alt.Axis(labelLimit=500)),
    color=alt.Color('value:Q', title="Value", scale=alt.Scale(
                range=["#917EF3", "#000000"]))
)

result = chart.configure(background='#ffffff'
        ).properties(
        width=700,
        height=400,
        title='Embeddings with 10 dimensions'
       ).configure_axis(
      labelFontSize=15,
      titleFontSize=12)

# Show the plot
result

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [16]:
# Function to generate the 2D plot
def generate_chart(df,xcol,ycol,lbl='on',color='basic',title=''):
    chart = alt.Chart(df).mark_circle(size=500).encode(
        x=
        alt.X(xcol,
              scale=alt.Scale(zero=False),
              axis=alt.Axis(labels=False, ticks=False, domain=False)
             ),
        y=
        alt.Y(ycol,
              scale=alt.Scale(zero=False),
              axis=alt.Axis(labels=False, ticks=False, domain=False)
             ),
        color= alt.value('#333293') if color == 'basic' else color,
        tooltip=['query']
    )

    if lbl == 'on':
        text = chart.mark_text(align='left', baseline='middle',dx=15, size=13,color='black').encode(text='query', color= alt.value('black'))
    else:
        text = chart.mark_text(align='left', baseline='middle',dx=10).encode()

    result = (chart + text).configure(background="#FDF7F0").properties(
        width=800,
        height=500,
        title=title
    ).configure_legend(orient='bottom', titleFontSize=18,labelFontSize=18)

    return result

In [17]:
# Reduce embeddings to 2 principal components to aid visualization
embeds_pc2 = get_pc(embeds, 2)

# Add the principal components to dataframe
df_pc2 = pd.concat([df, pd.DataFrame(embeds_pc2)], axis=1)

# Plot the 2D embeddings on a chart
df_pc2.columns = df_pc2.columns.astype(str)
generate_chart(df_pc2.iloc[:sample],'0','1',title='2D Embeddings')

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [18]:
# Define new query
new_query = "How can I find a taxi or a bus when the plane lands?"

# Get embeddings of the new query
new_query_embeds = get_embeddings([new_query], input_type="search_query")[0]

In [24]:

# Get embeddings of the new query
#new_query_embeds

In [20]:
# Calculate cosine similarity between the search query and existing queries
def get_similarity(target, candidates):
    # Turn list into array
    candidates = np.array(candidates)
    target = np.expand_dims(np.array(target),axis=0)

    # Calculate cosine similarity
    sim = cosine_similarity(target, candidates)
    sim = np.squeeze(sim).tolist()
    sort_index = np.argsort(sim)[::-1]
    sort_score = [sim[i] for i in sort_index]
    similarity_scores = zip(sort_index,sort_score)

    # Return similarity scores
    return similarity_scores

# Get the similarity between the search query and existing queries
similarity = get_similarity(new_query_embeds, embeds[:sample])

In [21]:
# View the top 5 articles
print('Query:')
print(new_query,'\n')

print('Most Similar Documents:')
for idx, sim in similarity:
    print(f'Similarity: {sim:.2f};', df.iloc[idx]['query'])


Query:
How can I find a taxi or a bus when the plane lands? 

Most Similar Documents:
Similarity: 0.37;  show me a list of ground transportation at boston airport
Similarity: 0.36;  what ground transportation is available in boston
Similarity: 0.33;  show me boston ground transportation
Similarity: 0.27;  show me the airlines that fly between toronto and denver
Similarity: 0.25;  which airlines fly from boston to washington dc via other cities
Similarity: 0.24;  of all airlines which airline has the most arrivals in atlanta
Similarity: 0.18;  i'd like the lowest fare from denver to pittsburgh
Similarity: 0.17;  show me round trip first class tickets from new york to miami
Similarity: 0.17;  i would like your rates between atlanta and boston on september third


In [22]:
# Create new dataframe and append new query
df_sem = df.copy()
df_sem.loc[len(df_sem.index)] = [new_query, new_query_embeds]

# Reduce embeddings dimension to 2
embeds_sem = np.array(df_sem['query_embeds'].tolist())
embeds_sem_pc2 = get_pc(embeds_sem, 2)

# Add the principal components to dataframe
df_sem_pc2 = pd.concat([df_sem, pd.DataFrame(embeds_sem_pc2)], axis=1)

In [23]:
# Create column for representing chart legend
df_sem_pc2['Source'] = 'Existing'
df_sem_pc2.at[len(df_sem_pc2)-1, 'Source'] = "New"

# Plot on a chart
df_sem_pc2.columns = df_sem_pc2.columns.astype(str)
selection = list(range(sample)) + [-1]
generate_chart(df_sem_pc2.iloc[selection],'0','1',color='Source',title='Semantic Search')

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [25]:
df.head()

,query,query_embeds
0,which airlines fly from boston to washington ...,"[0.02609253, 0.012168884, -0.008903503, 0.0114..."
1,show me the airlines that fly between toronto...,"[0.013801575, 0.017181396, -0.014984131, -0.00..."
2,show me round trip first class tickets from n...,"[0.02053833, -0.038482666, 0.061523438, 0.0099..."
3,i'd like the lowest fare from denver to pitts...,"[0.0016708374, 0.015625, -0.029022217, 0.03759..."
4,show me a list of ground transportation at bo...,"[0.037628174, -0.007888794, -0.0024662018, -0...."


In [26]:
# Embed the text for clustering
df['clustering_embeds'] = get_embeddings(df['query'].tolist(), input_type="clustering")
embeds = np.array(df['clustering_embeds'].tolist())

In [27]:
df.head()

,query,query_embeds,clustering_embeds
0,which airlines fly from boston to washington ...,"[0.02609253, 0.012168884, -0.008903503, 0.0114...","[0.02746582, 0.014625549, 0.010131836, -0.0153..."
1,show me the airlines that fly between toronto...,"[0.013801575, 0.017181396, -0.014984131, -0.00...","[-0.024978638, 0.02748108, -0.023117065, -0.02..."
2,show me round trip first class tickets from n...,"[0.02053833, -0.038482666, 0.061523438, 0.0099...","[0.020187378, -0.01953125, 0.08099365, -0.0265..."
3,i'd like the lowest fare from denver to pitts...,"[0.0016708374, 0.015625, -0.029022217, 0.03759...","[-0.0036182404, 0.034820557, -0.029449463, 0.0..."
4,show me a list of ground transportation at bo...,"[0.037628174, -0.007888794, -0.0024662018, -0....","[0.01676941, -0.00086069107, -0.013656616, -0...."


In [28]:
# Pick the number of clusters
n_clusters = 2

# Cluster the embeddings
kmeans_model = KMeans(n_clusters=n_clusters, random_state=0)
classes = kmeans_model.fit_predict(embeds).tolist()

# Store the cluster assignments
df_clust = df_pc2.copy()
df_clust['cluster'] = (list(map(str,classes)))

In [29]:
df_pc2

,query,query_embeds,0,1
0,which airlines fly from boston to washington ...,"[0.02609253, 0.012168884, -0.008903503, 0.0114...",0.073794,-0.252778
1,show me the airlines that fly between toronto...,"[0.013801575, 0.017181396, -0.014984131, -0.00...",0.005050,-0.316226
2,show me round trip first class tickets from n...,"[0.02053833, -0.038482666, 0.061523438, 0.0099...",0.195839,0.081208
3,i'd like the lowest fare from denver to pitts...,"[0.0016708374, 0.015625, -0.029022217, 0.03759...",0.111741,0.206023
4,show me a list of ground transportation at bo...,"[0.037628174, -0.007888794, -0.0024662018, -0....",-0.313675,-0.229604
...,...,...,...,...
86,what ground transportation is there in atlanta,"[0.014678955, 0.022964478, -0.020690918, -0.00...",-0.518085,0.111905
87,can i take a single airline from la to charlo...,"[-0.004180908, -0.031555176, -0.008918762, 0.0...",0.133057,-0.152838
88,what is the cost for a one way trip from pitt...,"[0.028198242, -0.015113831, 0.008026123, 0.009...",0.223587,0.161583
89,what ground transportation is available in ba...,"[0.019744873, 0.012123108, -0.03164673, -0.027...",-0.480022,0.191910


In [30]:
df_clust

,query,query_embeds,0,1,cluster
0,which airlines fly from boston to washington ...,"[0.02609253, 0.012168884, -0.008903503, 0.0114...",0.073794,-0.252778,0
1,show me the airlines that fly between toronto...,"[0.013801575, 0.017181396, -0.014984131, -0.00...",0.005050,-0.316226,0
2,show me round trip first class tickets from n...,"[0.02053833, -0.038482666, 0.061523438, 0.0099...",0.195839,0.081208,0
3,i'd like the lowest fare from denver to pitts...,"[0.0016708374, 0.015625, -0.029022217, 0.03759...",0.111741,0.206023,0
4,show me a list of ground transportation at bo...,"[0.037628174, -0.007888794, -0.0024662018, -0....",-0.313675,-0.229604,1
...,...,...,...,...,...
86,what ground transportation is there in atlanta,"[0.014678955, 0.022964478, -0.020690918, -0.00...",-0.518085,0.111905,1
87,can i take a single airline from la to charlo...,"[-0.004180908, -0.031555176, -0.008918762, 0.0...",0.133057,-0.152838,0
88,what is the cost for a one way trip from pitt...,"[0.028198242, -0.015113831, 0.008026123, 0.009...",0.223587,0.161583,0
89,what ground transportation is available in ba...,"[0.019744873, 0.012123108, -0.03164673, -0.027...",-0.480022,0.191910,1


In [32]:
# Plot on a chart
df_clust.columns = df_clust.columns.astype(str)
generate_chart(df_clust.iloc[:sample],'0','1',lbl='on',color='cluster',title='Clustering with 2 Clusters')


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)